In [1]:
import pandas as pd
import math
import torch
import pickle
import re

from yandexfreetranslate import YandexFreeTranslate
from tqdm import tqdm, trange
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, AutoConfig

import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
# os.environ['TORCH_USE_CUDA_DSA'] = '1'

# df = pd.read_json("hf://datasets/Vezora/Tested-143k-Python-Alpaca/143k-Tested-Python-Alpaca-Vezora.json")
# df.head(3)

/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/trinity/home/team16/workspace/.local/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


Ноутбук с переводом датасета для LORA на русский

In [16]:
# Не рбаотает, слишком млаенькое окно для перевода у модели
assert 2 == 3
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to(device)
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer.src_lang = "en_XX"

batch_size = 1
parts_number = math.ceil(part_to_translate_df.shape[0] / batch_size)

with torch.no_grad():
    for k in trange(parts_number):
        texts = list(part_to_translate_df['instruction'].iloc[k*batch_size:(k+1)*batch_size].values)
        encoded_texts = tokenizer(texts, return_tensors="pt").to(device)
        
        generated_tokens = model.generate(
            **encoded_texts,
            forced_bos_token_id=tokenizer.lang_code_to_id["ru_RU"]
        )
        
        tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)

AssertionError: 

In [4]:
# Работает но переводит код
assert 2 == 3
yt = YandexFreeTranslate(api = "ios")

translated_texts = {}
batch_size = 1
parts_number = math.ceil(part_to_translate_df.shape[0] / batch_size)
pattern = re.compile(r'(```.*?```|``.*?``|`.*?`)', re.DOTALL)

for k in trange(parts_number):
    text_to_translate = str(part_to_translate_df['instruction'].iloc[k*batch_size:(k+1)*batch_size].values[0])

    code_blocks = pattern.findall(text_to_translate)
    non_code_parts = pattern.split(text_to_translate)
    
    translated_parts = [yt.translate("en", "ru", part) for part in non_code_parts if part not in ["", " "]]
    
    translated_text = ''
    for i in range(len(translated_parts)):
        
        if "`" == text_to_translate[0]:
            translated_text += code_blocks[i]
        
        translated_text += translated_parts[i]
        
        if i < len(code_blocks):
            translated_text += code_blocks[i]

    translated_texts[k] = translated_text
    
    if k / 100 == k // 100:
        with open('translated_texts.pickle', 'wb') as handle:
            pickle.dump(translated_texts, handle, protocol=pickle.HIGHEST_PROTOCOL)

  1%|▏         | 137/10000 [02:25<2:55:08,  1.07s/it]


KeyboardInterrupt: 

In [1]:
import pandas as pd

df = pd.read_parquet("hf://datasets/iamtarun/python_code_instructions_18k_alpaca/data/train-00000-of-00001-8b6e212f3e1ece96.parquet")
df.head()

,instruction,input,output,prompt
0,Create a function to calculate the sum of a se...,"[1, 2, 3, 4, 5]",# Python code\ndef sum_sequence(sequence):\n ...,Below is an instruction that describes a task....
1,Generate a Python code for crawling a website ...,website: www.example.com \ndata to crawl: phon...,import requests\nimport re\n\ndef crawl_websit...,Below is an instruction that describes a task....
2,Create a Python list comprehension to get the ...,,"[x*x for x in [1, 2, 3, 5, 8, 13]]",Below is an instruction that describes a task....
3,Generate a python script to perform this action.,"Given a string, remove all the consecutive dup...",def remove_duplicates(string): \n result = ...,Below is an instruction that describes a task....
4,Write a python script to generates random numb...,,def generate_random_divisible_number():\n i...,Below is an instruction that describes a task....


In [2]:
df['prompt'][2]

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nCreate a Python list comprehension to get the squared values of a list [1, 2, 3, 5, 8, 13].\n\n### Input:\n\n\n### Output:\n[x*x for x in [1, 2, 3, 5, 8, 13]]'

In [23]:
part_to_translate_df = df.sample(10_000, random_state=42)

In [27]:
# Работает но переводит код
yt = YandexFreeTranslate(api = "ios")

translated_texts = {}
batch_size = 1
parts_number = math.ceil(part_to_translate_df.shape[0] / batch_size)
pattern = re.compile(r'(```.*?```|``.*?``|`.*?`)', re.DOTALL)

for k in trange(3445, parts_number):
    text_to_translate = str(part_to_translate_df['instruction'].iloc[k*batch_size:(k+1)*batch_size].values[0])

    code_blocks = pattern.findall(text_to_translate)
    non_code_parts = pattern.split(text_to_translate)
    
    translated_parts = [yt.translate("en", "ru", part) for part in non_code_parts if part not in ["", " "]]
    
    translated_text = ''
    for i in range(len(translated_parts)):
        
        if "`" == text_to_translate[0]:
            translated_text += code_blocks[i]
        
        translated_text += translated_parts[i]
        
        if i < len(code_blocks):
            translated_text += code_blocks[i]

    translated_texts[k] = translated_text
    
    if k / 100 == k // 100:
        with open('translated_texts.pickle', 'wb') as handle:
            pickle.dump(translated_texts, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('translated_texts.pickle', 'wb') as handle:
        pickle.dump(translated_texts, handle, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 6555/6555 [19:11<00:00,  5.69it/s]  


In [24]:
with open('translated_texts.pickle', 'rb') as handle:
    translated_texts = pickle.load(handle)

In [25]:
translated_texts_full = translated_texts

In [26]:
translated_texts_full_df = pd.Series(translated_texts_full)

In [27]:
part_to_translate_df = part_to_translate_df.reset_index(drop=True)

In [28]:
part_to_translate_df["instruction_ru"] = translated_texts_full_df
part_to_translate_df.head(3)

,instruction,input,output,prompt,instruction_ru
0,Generate a controller in Python Flask that ret...,Example Database table:\n\nTable name: users\n...,"from flask import Flask, jsonify\nimport sqlal...",Below is an instruction that describes a task....,"Создайте контроллер в Python Flask, который во..."
1,Generate a Python program to find the most fre...,"arr = [10, 8, 8, 9, 10, 11, 12, 12, 11]",# find the most frequent number in a given arr...,Below is an instruction that describes a task....,Сгенерируйте программу на Python для поиска на...
2,Modify the Python code below to create a funct...,"list=[1,2,3,4,5]",def sum_list(nums):\n total = 0\n for nu...,Below is an instruction that describes a task....,"Измените приведенный ниже код на Python, чтобы..."


In [19]:
part_to_translate_df.sample(5)

,instruction,input,output,prompt,instruction_ru
1067,Write a Python program to search a given strin...,"string = ""The quick brown fox"", substring = ""fox""","string = ""The quick brown fox""\nsubstring = ""f...",Below is an instruction that describes a task....,Напишите программу на Python для поиска в зада...
2606,Create a Python function to check if two strin...,,"def is_anagram(str1, str2):\n list_str1 = l...",Below is an instruction that describes a task....,"Создайте функцию Python, чтобы проверить, явля..."
3269,Create a Python script to detect if a certain ...,"items = [1,2,3,4,5]","def detect_item(items, item):\n if item in it...",Below is an instruction that describes a task....,"Создайте скрипт на Python, чтобы определить, е..."
5503,Design a class in Python to represent a Rectan...,,"class Rectangle:\n def __init__(self, length,...",Below is an instruction that describes a task....,Разработайте класс в Python для представления ...
7138,Write a Python program that finds all numbers ...,"""15 apples, 4 bananas and 6 oranges""",import re\n\ndef find_numbers(s):\n numbers...,Below is an instruction that describes a task....,"Написать программу на Python, которая находит ..."


In [29]:
string_format = 'Ниже приведена инструкция, описывающая задачу. Напишите ответ, который соответствующим образом завершает запрос.\n\n### Инструкция:\n{instraction}\n\n### Входные данные:\n{input_code}'

part_to_translate_df['prompt_ru'] = part_to_translate_df.apply(lambda row: string_format.format(
    instraction=row['instruction_ru'],
    input_code=row['input'],
    output_code=row['output']
), axis=1)

In [30]:
from datasets import Dataset, load_dataset

df_to_hugging = part_to_translate_df[["instruction_ru", "input", "output"]]
df_to_hugging.columns = ['instruction', 'input', 'output']
dataset_train = Dataset.from_pandas(df_to_hugging)

In [31]:
dataset_train.save_to_disk('iamtarun/python_code_instructions_18k_alpaca_ru')

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

In [34]:
dataset_train_prompt = Dataset.from_pandas(part_to_translate_df[["prompt_ru"]])
dataset_train_prompt.save_to_disk('python_code_instructions_18k_alpaca_ru_prompt')

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]